In [7]:
import io
import re
import string
import tensorflow as tf
import tqdm
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px 
import spacy
import fr_core_news_sm
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten

import warnings
warnings.filterwarnings('ignore')

In [8]:
p = r'C:\Users\david\Downloads'
f = os.path.join(p, '', 'df_nlp.csv')
df = pd.read_csv(f)
df.head(2)

,Unnamed: 0,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,note,date,appreciations,commentaire,com_process,tokenized,filtre_sentiments,sentimentest,sentiments_final,com_final
0,0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,4.0,30 novembre 2019,85,Roadtrip désertique dans l'Ouest Américain en ...,roadtrip désertique dans l'ouest américain en ...,"[roadtrip, désertique, ouest, américain, compa...","[furieux, plaisir]","['colère', 'joie']",colère joie,roadtrip désertique dans l' ouest américain en...
1,1,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,1271915,319761,5.0,03 mars 2017,68,"De temps en temps, j'aime bien revenir vers l'...",de temps en temps j'aime bien revenir vers l'...,"[temps, temps, , aime, bien, revenir, oeuvre,...","[humour, bonheur, plaisir, rire, angoisse, pla...","['joie', 'joie', 'joie', 'joie', 'peur', 'joie']",joie joie joie joie peur joie,de temps en temps j' aime bien revenir vers ...


In [9]:
df.shape

(62124, 20)

### Remove NaN values

In [10]:
def test_null_values(df, print_=False, clean_nan=False):
    df_ = df.copy()
    for i, col in enumerate(df_.select_dtypes('object')):
        df_[col] = df_[col].apply(lambda x:np.nan if x=='' else x)
    if print_:
        print(df_.isna().sum()) 
    index_ = df_[pd.isnull(df_).any(1)].index
    if clean_nan:
        mask_nan = df_.index.isin(index_)
        df_ = df_[~mask_nan]
        df_ = df_.reset_index().drop(['index'], axis=1)
    return df_

In [12]:
df = test_null_values(df, print_=True, clean_nan=True)

Unnamed: 0               0
book_id                  0
book_nb_comm             0
title                    0
name                  1796
surname                  0
tags                     0
img_url                  0
comm_id                  0
user_id                  0
note                  3670
date                     0
appreciations            0
commentaire              4
com_process              4
tokenized                0
filtre_sentiments        0
sentimentest             0
sentiments_final     12579
com_final                4
dtype: int64


### Creation d'une colonne 'nombres de tags dans la colonne tags'

In [366]:
def create_tags(df, col):
    df['len_tags'] = df[col].apply(lambda x:len(x.split(',')))
    df['tags_0'] = df[col].apply(lambda x:x.split(',')[0])
    return df

In [367]:
df = create_tags(df, 'tags')

### Creation de n colonnes en fonction de n tags 

In [368]:
def create_tags_columns(df):
    for i, el in enumerate(df['len_tags'].tolist()):
        j = 0
        while j < el:
            globals() [f"tags_{j+1}"] = df.loc[i, 'tags'].split(',')[j]
            df.loc[i, f"tags_{j+1}"] = globals() [f"tags_{j+1}"]
            j +=1 
    return df

In [369]:
df = create_tags_columns(df)

In [370]:
df.head(1)

,Unnamed: 0,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,...,tags_13,tags_14,tags_15,tags_16,tags_17,tags_18,tags_19,tags_20,title_test,tags_21
0,0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,...,nature writing,grands espaces,écologie,montagnes,états-unis,amérique,littérature américaine,20ème siècle,"Le gang de la clef à molette (Ne meurs pas, ô ...",NaN


In [371]:
df.columns

Index(['Unnamed: 0', 'book_id', 'book_nb_comm', 'title', 'name', 'surname',
       'tags', 'img_url', 'comm_id', 'user_id', 'note', 'date',
       'appreciations', 'commentaire', 'com_process', 'tokenized',
       'filtre_sentiments', 'sentimentest', 'sentiments_final', 'com_final',
       'len_tags', 'tags_0', 'tags_1', 'tags_2', 'tags_3', 'tags_4', 'tags_5',
       'tags_6', 'tags_7', 'tags_8', 'tags_9', 'tags_10', 'tags_11', 'tags_12',
       'tags_13', 'tags_14', 'tags_15', 'tags_16', 'tags_17', 'tags_18',
       'tags_19', 'tags_20', 'title_test', 'tags_21'],
      dtype='object')

In [14]:
df.shape

(45695, 20)

## 1st Step : supprimer les `sentiments` des commentaires et les remplacer par `<MASK>`

### Fonction qui transforme un dictionnaire dans un fichier text en un veritable dictionnaire

In [13]:
path_text = './Dictionnaire_emotions/'

def text_to_dict(path_text):

    p = './Dictionnaire_emotions/'
    general_emotions = []
    first_ind, last_ind = [],[]
    text = []
    with open(p+'dictionnaire_emotion_rebecca.txt', 'r') as f:
        for i,line in enumerate(f):
            if '[' in line:
                text.append(line.split('[')[1][1:-3])
                general_emotions.append(line[2:].split("'")[0])
                first_ind.append(i)
            elif ']' in line:
                text.append(line.split(']')[0][3:-1])
                last_ind.append(i)
            else:
                text.append(line.split()[0][1:-2])

    d = {}
    for el in general_emotions:
        d[el] = []

    for i,j,k in zip(first_ind, last_ind, range(len(first_ind))):
        d[general_emotions[k]] = text[i:j]

    return d
            
emotions = text_to_dict(path_text)
emotions

{'joie': ['joyeux',
  'paradis',
  'magique',
  'content',
  'épanouir',
  'qualité',
  'épanouie',
  'magnifique',
  'beau',
  'avantageux',
  'béat',
  'heureux',
  'bien',
  'enchanter',
  'délicieux',
  'enjoué',
  'drôle',
  'rire',
  'rigoler',
  'allégresse',
  'distraire',
  'amuser',
  'détendre',
  'divertir',
  'jouer',
  'distraction',
  'amusant',
  'amusement',
  'épanouissement',
  'avantage',
  'béatitude',
  'bienfait',
  'bonheur',
  'consolation',
  'contentement',
  'délice',
  'satisfait',
  'satisfaisant',
  'enchantement',
  'brillant',
  'briller',
  'réussite',
  'réussir',
  'engouement',
  'charm',
  'enthousiasme',
  'entrain',
  'euphorie',
  'exaltation',
  'extase',
  'félicité',
  'fierté',
  'gaieté',
  'hilarité',
  'ivresse',
  'jouissance',
  'régal',
  'réjouissance',
  'ravissement',
  'rayonnement',
  'rigolade',
  'vibrant',
  'satisfaction',
  'sourire',
  'attendrie',
  'attendrir',
  'égaux',
  'captivant',
  'parfait',
  'apaisé',
  'doux',
 

### Dictionnaire de Rebecca

In [15]:
for k,v in emotions.items():
    print(k,len(v))

joie 127
colere 105
surprise 34
tristesse 75
peur 59
confiance 52
degout 48
reflexion 33


### Dictionnaire d'Antoine

In [16]:
import os
p = r'C:\Users\david\Downloads'
f = os.path.join(p, '', 'lexique_emotions (1).csv')
emotions_antoine = pd.read_csv(f, sep=';')
emotions_antoine

,sentiment,mot
0,colère,rage
1,colère,fureur
2,colère,insupportable
3,colère,exaspération
4,colère,exaspérant
...,...,...
431,tristesse,pénible
432,tristesse,désenchanté
433,tristesse,émouvant
434,tristesse,émouvante


### Fonction qui crée une liste de tous les sentiments soit par un dictionnaire soit par un csv

In [17]:
def list_all_emotions_dict(dictionnaire):
    l = [(k+' '+' '.join(v)).split() for k,v in dictionnaire.items()]
    flatten_l = [el for element in l for el in element]
    return flatten_l

In [18]:
def list_all_emotions_csv(df, cols):
    liste = []
    for el in cols:
        liste += list(df[el].unique())
    return liste

In [19]:
emotions_antoine.groupby('sentiment')['mot'].count()

sentiment
colère       73
confiance    38
dégoût       55
honte        17
joie         56
peur         52
réflexion    15
surprise     64
tristesse    66
Name: mot, dtype: int64

In [24]:
list_all_emotion_rebecca = list_all_emotions_dict(emotions)
list_all_emotion_rebecca

['joie',
 'joyeux',
 'paradis',
 'magique',
 'content',
 'épanouir',
 'qualité',
 'épanouie',
 'magnifique',
 'beau',
 'avantageux',
 'béat',
 'heureux',
 'bien',
 'enchanter',
 'délicieux',
 'enjoué',
 'drôle',
 'rire',
 'rigoler',
 'allégresse',
 'distraire',
 'amuser',
 'détendre',
 'divertir',
 'jouer',
 'distraction',
 'amusant',
 'amusement',
 'épanouissement',
 'avantage',
 'béatitude',
 'bienfait',
 'bonheur',
 'consolation',
 'contentement',
 'délice',
 'satisfait',
 'satisfaisant',
 'enchantement',
 'brillant',
 'briller',
 'réussite',
 'réussir',
 'engouement',
 'charm',
 'enthousiasme',
 'entrain',
 'euphorie',
 'exaltation',
 'extase',
 'félicité',
 'fierté',
 'gaieté',
 'hilarité',
 'ivresse',
 'jouissance',
 'régal',
 'réjouissance',
 'ravissement',
 'rayonnement',
 'rigolade',
 'vibrant',
 'satisfaction',
 'sourire',
 'attendrie',
 'attendrir',
 'égaux',
 'captivant',
 'parfait',
 'apaisé',
 'doux',
 'tranquillement',
 'équilibré',
 'pure',
 'reposer',
 'sérénité',
 'ac

In [25]:
list_all_emotion_antoine = list_all_emotions_csv(emotions_antoine, ['sentiment', 'mot'])
list_all_emotion_antoine

['colère',
 'confiance',
 'dégoût',
 'honte',
 'joie',
 'peur',
 'réflexion',
 'surprise',
 'tristesse',
 'rage',
 'fureur',
 'insupportable',
 'exaspération',
 'exaspérant',
 'exaspérante',
 'agacement',
 'agaçant',
 'agaçante',
 'contrariété',
 'courroux',
 'âpre',
 'dureté',
 'brutalité',
 'brutal',
 'brutale',
 'froideur',
 'sauvage',
 'vengée',
 'vengé',
 'cruel',
 'cruelle',
 'cruauté',
 'vengeance',
 'ressentiment',
 'rancœur',
 'grogne',
 'hargne',
 'irritation',
 'irritée',
 'irrité',
 'massacre',
 'fâché',
 'furieux',
 'carnage',
 'furibond',
 'révoltant',
 'révoltante',
 'révolte',
 'révolté',
 'révoltée',
 'outragé',
 'offensée',
 'offensé',
 'enragé',
 'indigné',
 'violent',
 'violente',
 'violence',
 'agressive',
 'agressif',
 'vexé',
 'exaspérée',
 'exaspéré',
 'frustré',
 'cynique',
 'animosité',
 'frustrée',
 'frustration',
 'mécontent',
 'mécontentement',
 'violemment',
 'agacé',
 'agacée',
 'furax',
 'énervé',
 'énervée',
 'énervement',
 'scandalisé',
 'scandaleux',


### Mot unique dans le dictionnaire de Rebecca

In [26]:
def unique_word_rebecca(liste1, liste2):
    s = []
    for el_rebecca in liste1:
        if el_rebecca not in liste2:
            s.append(el_rebecca)
    return s

In [27]:
unique_rebecca = unique_word_rebecca(list_all_emotion_rebecca, list_all_emotion_antoine)
unique_rebecca

['épanouir',
 'qualité',
 'épanouie',
 'avantageux',
 'béat',
 'heureux',
 'bien',
 'enchanter',
 'délicieux',
 'rigoler',
 'distraire',
 'amuser',
 'détendre',
 'divertir',
 'jouer',
 'distraction',
 'amusant',
 'épanouissement',
 'avantage',
 'béatitude',
 'bienfait',
 'consolation',
 'délice',
 'satisfaisant',
 'enchantement',
 'brillant',
 'briller',
 'réussite',
 'réussir',
 'engouement',
 'charm',
 'enthousiasme',
 'entrain',
 'exaltation',
 'fierté',
 'hilarité',
 'ivresse',
 'jouissance',
 'régal',
 'réjouissance',
 'ravissement',
 'rayonnement',
 'sourire',
 'attendrie',
 'attendrir',
 'égaux',
 'captivant',
 'parfait',
 'doux',
 'pure',
 'reposer',
 'sérénité',
 'accalmie',
 'égalité',
 'apaisement',
 'équilibre',
 'bien-être',
 'décontraction',
 'douceur',
 'patience',
 'placidité',
 'pureté',
 'quiétude',
 'repos',
 'conquérir',
 'désirer',
 'désireux',
 'désir',
 'aimer',
 'adoration',
 'affect',
 'affection',
 'altruisme',
 'altruiste',
 'ardeur',
 'attraction',
 'béguin'

In [28]:
'épanouir' in list_all_emotion_antoine

False

In [29]:
for el in unique_rebecca:
    print(el in list_all_emotion_antoine)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

### Mot unique dans le dictionnaire d'Antoine

In [30]:
def unique_word_antoine(liste1, liste2):
    s = []
    for el_antoine in liste1:
        if el_antoine not in liste2:
            s.append(el_antoine)
    return s

In [31]:
unique_antoine = unique_word_antoine(list_all_emotion_antoine, list_all_emotion_rebecca)
unique_antoine

['réflexion',
 'insupportable',
 'exaspérant',
 'exaspérante',
 'agaçant',
 'agaçante',
 'courroux',
 'âpre',
 'brutalité',
 'brutal',
 'vengée',
 'vengé',
 'cruelle',
 'rancœur',
 'grogne',
 'irritée',
 'irrité',
 'massacre',
 'fâché',
 'carnage',
 'furibond',
 'révoltant',
 'révoltante',
 'révolte',
 'révolté',
 'révoltée',
 'outragé',
 'offensée',
 'offensé',
 'enragé',
 'indigné',
 'violente',
 'agressive',
 'vexé',
 'exaspérée',
 'frustré',
 'cynique',
 'frustrée',
 'frustration',
 'mécontent',
 'violemment',
 'agacé',
 'agacée',
 'furax',
 'énervé',
 'énervée',
 'énervement',
 'scandalisé',
 'provocante',
 'provocant',
 'attachant',
 'attachante',
 'attachants',
 'amical',
 'amicale',
 'intéressant',
 'intéressante',
 'apaisant',
 'apaisante',
 'foi',
 'serein',
 'sereine',
 'optimiste',
 'tranquille',
 'admirable',
 'optimisme',
 'détente',
 'détendu',
 'bienveillance',
 'touchant',
 'touchante',
 'touché',
 'touchée',
 'écoeuré',
 'écoeurée',
 'dégoutante',
 'crasse',
 'crasseu

In [32]:
def verify_missing_words(liste1, liste2, print_=False):
    count = 0
    for el in liste1:
        if el not in liste2:
            if print_:
                print(el)
            count +=1
    return count

In [33]:
verify_missing_words(unique_antoine, list_all_emotion_rebecca)

296

### Verdict !

In [34]:
print("mot dans la liste de rebecca uniquement :", len(unique_rebecca))
print("mot dans la liste d'antoine uniquement  :", len(unique_antoine))

mot dans la liste de rebecca uniquement : 392
mot dans la liste d'antoine uniquement  : 296


In [35]:
print("nombre total de mots dans la liste de rebecca :", len(list_all_emotion_rebecca))
print("nombre total de mots dans la liste d'antoine  :", len(list_all_emotion_antoine))

nombre total de mots dans la liste de rebecca : 541
nombre total de mots dans la liste d'antoine  : 435


### Verification des mots de sentiment manquants

In [36]:
df_sample = df.iloc[:10]

In [37]:
def remove_sentiment(df, list_all_emotion, print_=True, BERT=False):
    i,j = 0, len(df)
    word_s, ind = [], []
    df['commentaire_test'] = df['commentaire']
    df['sentiments'] = [[] for i in range(df.shape[0])]
    
    while i<j:
        for el in list_all_emotion:
            try:
                if el in df.loc[i, 'commentaire_test']:
                    word_s.append(el)
                    ind.append(i)
        
                    if BERT:
                        df.loc[i, 'commentaire_BERT'] = df.loc[i, 'commentaire_test'].replace(el, '<MASK>')
                    else:
                        df.loc[i, 'commentaire_test'] = df.loc[i, 'commentaire_test'].replace(el, '')
                        
                    df.loc[i,'sentiments'].append(el)
                    
            except:
                pass
        i+=1
    try:
        if BERT:
            df['commentaire_BERT'] = df['commentaire_BERT'].apply(lambda x:x if len(x)>1 else '')
        else:
            df['commentaire_test'] = df['commentaire_test'].apply(lambda x:x if len(x)>1 else '')
    except:
        pass
        
    dictionnaire = {}
    
    for el in ind:
        if el not in dictionnaire:
            dictionnaire[el] = 1
        else:
            dictionnaire[el] += 1
            
    if print_:
        print("index :\n")
        for k,v in dictionnaire.items():
            print("index  :",k,"nombre :",v,"sentiments")
        print("\nnombre de sentiments retirés :\n")
        print(len(word_s))
        
    return df, word_s, ind

In [38]:
df_sample, word_s, ind = remove_sentiment(df_sample, list_all_emotion_antoine)
df_sample['commentaire_test'][0]

index :

index  : 0 nombre : 2 sentiments
index  : 1 nombre : 9 sentiments
index  : 2 nombre : 4 sentiments
index  : 3 nombre : 7 sentiments
index  : 4 nombre : 17 sentiments
index  : 5 nombre : 4 sentiments
index  : 6 nombre : 3 sentiments
index  : 7 nombre : 5 sentiments
index  : 8 nombre : 11 sentiments
index  : 9 nombre : 6 sentiments

nombre de sentiments retirés :

68


"Roadtrip désertique dans l'Ouest Américain en compagnie de 4 gentils fous  qui ne pensent qu'à une seule chose : le Sabotage! le moteur de cette épopée épique et déjantée est la lutte contre le développement du capitalisme qui se propage de manière frénétique dans toute l'Amérique, même en plein désert...L'écriture est dense, voire très dense et pas forcément facile de premier abord. Il m'a fallu entre 50 et 80 pages pour réellement rentrer dans le roman. On ne peut ensuite plus reposer le livre tant le rythme entre course-poursuites et dynamitages est effréné. Publié en 1975 et pourtant toujours autant d'actualité à propos du capitalisme et de son immense impact écologique, Edward ABBEY nous prouve ici qu'il est un visionnaire écologiste et aurait quelque part rêvé être l'un de ses propres personnages du roman.Un grand merci à Flocava1 de m'avoir offert ce livre et donné le  de découvrir Edward ABBEY que je ne connaissais pas."

In [39]:
def remove_punctuations(df, col, lemma=None):
    import string
    corpus = df[col].tolist()
    nlp = fr_core_news_sm.load()
    comms = []
    for el in corpus:
        out = el.replace("'", ' ')
        out = el.replace(".", ' ')
        out = out.translate(str.maketrans(' ',' ',string.punctuation))
        out = out.replace(':', '').replace('\n', ' ').strip()
        if lemma:
#            out = [token.lemma_ for token in nlp(out) if (token.lemma_ not in stopwords) and (len(token) > 1) and (token.text not in stopwords) or (token.text=='x' or token.text=='y' or token.text=='i')]
            out = [token.lemma_ for token in nlp(out) if  (len(token) > 1)]
            out = ' '.join(out)
        comms.append(out)
    df["commentaire_clean"] = pd.Series(comms)
    return df["commentaire_clean"]

In [40]:
df_sample["commentaire_clean_lemma"] = remove_punctuations(df_sample, 'commentaire', lemma=True)
df_sample.head(2)

,Unnamed: 0,book_id,book_nb_comm,title,name,surname,tags,img_url,comm_id,user_id,...,com_process,tokenized,filtre_sentiments,sentimentest,sentiments_final,com_final,commentaire_test,sentiments,commentaire_clean,commentaire_clean_lemma
0,0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,...,roadtrip désertique dans l'ouest américain en ...,"[roadtrip, désertique, ouest, américain, compa...","[furieux, plaisir]","['colère', 'joie']",colère joie,roadtrip désertique dans l' ouest américain en...,Roadtrip désertique dans l'Ouest Américain en ...,"[furieux, plaisir]",roadtrip désertique dans louest Américain en c...,roadtrip désertique dans louest Américain en c...
1,1,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,"roman,romans policiers et polars,thriller,colo...",/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,1271915,319761,...,de temps en temps j'aime bien revenir vers l'...,"[temps, temps, , aime, bien, revenir, oeuvre,...","[humour, bonheur, plaisir, rire, angoisse, pla...","['joie', 'joie', 'joie', 'joie', 'peur', 'joie']",joie joie joie joie peur joie,de temps en temps j' aime bien revenir vers ...,"De temps en temps, j'aime bien revenir vers l'...","[réflexion, rage, massacre, touchant, rire, bo...",de temps en temps jaim bien revenir vers loeuv...,de temps en temps jaim bien revenir vers loeuv...


In [41]:
df_sample, word_s, ind = remove_sentiment(df_sample, list_all_emotion_antoine)

index :

index  : 0 nombre : 2 sentiments
index  : 1 nombre : 9 sentiments
index  : 2 nombre : 4 sentiments
index  : 3 nombre : 7 sentiments
index  : 4 nombre : 17 sentiments
index  : 5 nombre : 4 sentiments
index  : 6 nombre : 3 sentiments
index  : 7 nombre : 5 sentiments
index  : 8 nombre : 11 sentiments
index  : 9 nombre : 6 sentiments

nombre de sentiments retirés :

68


In [42]:
'frénétique' in list_all_emotion_antoine

False

In [43]:
df_sample,_,_= remove_sentiment(df_sample, list_all_emotion_rebecca, BERT=True, print_=False)
df_sample['commentaire_BERT'][0]

"Roadtrip désertique dans l'Ouest Américain en compagnie de 4 gentils fous <MASK> qui ne pensent qu'à une seule chose : le Sabotage! le moteur de cette épopée épique et déjantée est la lutte contre le développement du capitalisme qui se propage de manière frénétique dans toute l'Amérique, même en plein désert...L'écriture est dense, voire très dense et pas forcément facile de premier abord. Il m'a fallu entre 50 et 80 pages pour réellement rentrer dans le roman. On ne peut ensuite plus reposer le livre tant le rythme entre course-poursuites et dynamitages est effréné. Publié en 1975 et pourtant toujours autant d'actualité à propos du capitalisme et de son immense impact écologique, Edward ABBEY nous prouve ici qu'il est un visionnaire écologiste et aurait quelque part rêvé être l'un de ses propres personnages du roman.Un grand merci à Flocava1 de m'avoir offert ce livre et donné le plaisir de découvrir Edward ABBEY que je ne connaissais pas."

#### On verifie en fonction du dictionnaire de Rebecca

In [44]:
pd.Series(word_s).value_counts()

foi             4
plaisir         4
sauvage         4
furieux         3
réflexion       3
intérêt         2
improbable      2
joie            2
dommage         2
attachant       2
doute           2
beau            2
rage            2
joyeux          1
heureuse        1
perturbée       1
agacé           1
intéressant     1
ennui           1
abattu          1
haletant        1
enthousiaste    1
accablé         1
stress          1
tristesse       1
tension         1
optimiste       1
amitié          1
sale            1
déçu            1
massacre        1
touchant        1
rire            1
bonheur         1
angoisse        1
intelligent     1
équilibré       1
drôle           1
immonde         1
magistrale      1
rare            1
larme           1
confiance       1
révolte         1
violence        1
bienveillant    1
espoir          1
dtype: int64

In [45]:
verify_missing_words(word_s, list_all_emotion_rebecca, print_=True)

réflexion
massacre
touchant
intelligent
foi
doute
magistrale
rare
dommage
réflexion
révolte
foi
immonde
sale
heureuse
tension
stress
improbable
haletant
doute
abattu
attachant
attachant
optimiste
perturbée
réflexion
agacé
intéressant
foi
enthousiaste
accablé
dommage
foi
improbable


34

In [46]:
verify_missing_words(word_s, list_all_emotion_antoine)

0

In [47]:
def verify_missing_words(liste1, liste2, print_=False):
    count = 0
    for el in liste1:
        if print_:
            print(el in liste2)
        count +=1
    return count

In [48]:
'déjanté' in list_all_emotion_rebecca

False

In [49]:
df_sample['commentaire_test'][1]

'De temps en temps, j\'aime bien revenir vers l\'oeuvre de mon auteur favori, Edward Abbey. Son combat pour la sauvegarde des milieux naturels fut reconnu aux Etats Unis et a toujours valeur d\'exemple car en France, resurgissent ici et là des projets de fracturation du sol et du sous-sol, des projets miniers de tous genres menaçant l\'intégrité de paysages sublimes .Et utiliser l\'humour comme vecteur de réflexion c\'est un exercice subtile et intelligent qu\'a réussi E. Abbey .De temps en temps, reparler de son oeuvre me semble salutaire !Alors, dans ce roman épique, on va suivre le gang de doux dingues prêts à tout pour déjouer des projets qu\'ils jugent assassins pour le milieu naturel . Et là, on est parti pour s\'octroyer un moment de lecture de pure folie et de pur bonheur.Bien sûr, le personnage principal, le plus déjanté, le plus touchant aussi c\'est Hayduke, ancien combattant au viet-nam.Mais ici, fiction et réalité se mêlent : on murmure dans les canyons qu\' Abbey ait pris

In [50]:
df_sample['commentaire'][1]

'De temps en temps, j\'aime bien revenir vers l\'oeuvre de mon auteur favori, Edward Abbey. Son combat pour la sauvegarde des milieux naturels fut reconnu aux Etats Unis et a toujours valeur d\'exemple car en France, resurgissent ici et là des projets de fracturation du sol et du sous-sol, des projets miniers de tous genres menaçant l\'intégrité de paysages sublimes .Et utiliser l\'humour comme vecteur de réflexion c\'est un exercice subtile et intelligent qu\'a réussi E. Abbey .De temps en temps, reparler de son oeuvre me semble salutaire !Alors, dans ce roman épique, on va suivre le gang de doux dingues prêts à tout pour déjouer des projets qu\'ils jugent assassins pour le milieu naturel . Et là, on est parti pour s\'octroyer un moment de lecture de pure folie et de pur bonheur.Bien sûr, le personnage principal, le plus déjanté, le plus touchant aussi c\'est Hayduke, ancien combattant au viet-nam.Mais ici, fiction et réalité se mêlent : on murmure dans les canyons qu\' Abbey ait pris

In [51]:
df_sample['commentaire_test'][0]

"Roadtrip désertique dans l'Ouest Américain en compagnie de 4 gentils fous furieux qui ne pensent qu'à une seule chose : le Sabotage! le moteur de cette épopée épique et déjantée est la lutte contre le développement du capitalisme qui se propage de manière frénétique dans toute l'Amérique, même en plein désert...L'écriture est dense, voire très dense et pas forcément facile de premier abord. Il m'a fallu entre 50 et 80 pages pour réellement rentrer dans le roman. On ne peut ensuite plus reposer le livre tant le rythme entre course-poursuites et dynamitages est effréné. Publié en 1975 et pourtant toujours autant d'actualité à propos du capitalisme et de son immense impact écologique, Edward ABBEY nous prouve ici qu'il est un visionnaire écologiste et aurait quelque part rêvé être l'un de ses propres personnages du roman.Un grand merci à Flocava1 de m'avoir offert ce livre et donné le plaisir de découvrir Edward ABBEY que je ne connaissais pas."

In [52]:
'furieux' in list_all_emotion

NameError: name 'list_all_emotion' is not defined

In [53]:
df['commentaire'][0]

"Roadtrip désertique dans l'Ouest Américain en compagnie de 4 gentils fous furieux qui ne pensent qu'à une seule chose : le Sabotage! le moteur de cette épopée épique et déjantée est la lutte contre le développement du capitalisme qui se propage de manière frénétique dans toute l'Amérique, même en plein désert...L'écriture est dense, voire très dense et pas forcément facile de premier abord. Il m'a fallu entre 50 et 80 pages pour réellement rentrer dans le roman. On ne peut ensuite plus reposer le livre tant le rythme entre course-poursuites et dynamitages est effréné. Publié en 1975 et pourtant toujours autant d'actualité à propos du capitalisme et de son immense impact écologique, Edward ABBEY nous prouve ici qu'il est un visionnaire écologiste et aurait quelque part rêvé être l'un de ses propres personnages du roman.Un grand merci à Flocava1 de m'avoir offert ce livre et donné le plaisir de découvrir Edward ABBEY que je ne connaissais pas."

### Test sur tout le DataFrame

In [54]:
new_df , word_s, ind = remove_sentiment(df, list_all_emotion_antoine)

index :

index  : 0 nombre : 2 sentiments
index  : 1 nombre : 9 sentiments
index  : 2 nombre : 4 sentiments
index  : 3 nombre : 7 sentiments
index  : 4 nombre : 17 sentiments
index  : 5 nombre : 4 sentiments
index  : 6 nombre : 3 sentiments
index  : 7 nombre : 5 sentiments
index  : 8 nombre : 11 sentiments
index  : 9 nombre : 6 sentiments
index  : 10 nombre : 6 sentiments
index  : 11 nombre : 9 sentiments
index  : 12 nombre : 7 sentiments
index  : 13 nombre : 6 sentiments
index  : 14 nombre : 8 sentiments
index  : 15 nombre : 1 sentiments
index  : 16 nombre : 3 sentiments
index  : 17 nombre : 5 sentiments
index  : 18 nombre : 6 sentiments
index  : 19 nombre : 1 sentiments
index  : 20 nombre : 1 sentiments
index  : 21 nombre : 8 sentiments
index  : 22 nombre : 9 sentiments
index  : 23 nombre : 6 sentiments
index  : 24 nombre : 2 sentiments
index  : 25 nombre : 1 sentiments
index  : 26 nombre : 3 sentiments
index  : 27 nombre : 2 sentiments
index  : 28 nombre : 11 sentiments
index  : 29 

index  : 25542 nombre : 2 sentiments
index  : 25543 nombre : 27 sentiments
index  : 25544 nombre : 2 sentiments
index  : 25545 nombre : 1 sentiments
index  : 25546 nombre : 5 sentiments
index  : 25547 nombre : 2 sentiments
index  : 25548 nombre : 1 sentiments
index  : 25549 nombre : 5 sentiments
index  : 25550 nombre : 13 sentiments
index  : 25551 nombre : 7 sentiments
index  : 25552 nombre : 10 sentiments
index  : 25553 nombre : 4 sentiments
index  : 25554 nombre : 5 sentiments
index  : 25555 nombre : 2 sentiments
index  : 25556 nombre : 1 sentiments
index  : 25557 nombre : 2 sentiments
index  : 25558 nombre : 8 sentiments
index  : 25559 nombre : 5 sentiments
index  : 25560 nombre : 6 sentiments
index  : 25561 nombre : 2 sentiments
index  : 25562 nombre : 3 sentiments
index  : 25563 nombre : 1 sentiments
index  : 25564 nombre : 8 sentiments
index  : 25565 nombre : 21 sentiments
index  : 25566 nombre : 1 sentiments
index  : 25567 nombre : 4 sentiments
index  : 25568 nombre : 6 sentimen

In [55]:
new_df.shape

(45695, 22)

In [56]:
new_df["commentaire_clean_lemma"] = remove_punctuations(new_df, 'commentaire', lemma=True)

In [57]:
new_df["commentaire_clean"] = remove_punctuations(new_df, 'commentaire', lemma=False)

In [58]:
df__,_,_= remove_sentiment(df_sample, list_all_emotion_rebecca, BERT=True, print_=False)
new_df['commentaire_BERT'] = df__['commentaire_BERT']

In [59]:
new_df.to_csv('last_version.csv', index=False)